In [1]:
import numpy as np
import pandas as pd




In [2]:
# Define the endmember distributions from Keller et al., 2024 (Organic Geochemistry)
gdgt_distribution = np.array([[0.31, 0.18, 0.40, 0.06, 0.05, 0.01], #meth, modify iGDGT-4 to account for minor presence  
    [0.46, 0.07, 0.03, 0.00, 0.00, 0.43]]) #plank

In [3]:
#import sample data from Keller et al., 2024
fn = '../data/AC_relative_abundance_bulk.xlsx'
sample_data= pd.read_excel(fn, sheet_name='gdgt_rel_abund')
sample_data



,Name,G0,G1,G2,G3,G4,Cren,G5
0,S1,0.389506,0.097763,0.157369,0.042605,0.000274,0.306266,0.006217
1,S4,0.481303,0.112187,0.158765,0.021725,0.003057,0.219767,0.003197
2,S5,0.435868,0.157116,0.226899,0.031261,0.002502,0.144034,0.002321
3,S7,0.416644,0.092921,0.127931,0.056481,0.008502,0.292885,0.004636
4,S9,0.302591,0.107944,0.248903,0.116816,0.015371,0.204646,0.003729
5,S10,0.272736,0.108790,0.270738,0.134849,0.019270,0.190468,0.003149
6,S11,0.251405,0.116905,0.282700,0.168619,0.026520,0.151434,0.002418


In [4]:

# Convert sample data to numpy array
sample_distributions = sample_data.iloc[:, 1:].to_numpy()

# Generate linear combinations
fractions = np.linspace(0, 1, 101)  # Fractions from 0% to 100% in 1% increments
linear_combinations = np.array([f * meth + (1 - f) * planktonic for f in fractions])

# Calculate the best match for each sample
best_matches = []
for sample in sample_distributions:
    mse = np.mean((linear_combinations - sample) ** 2, axis=1)
    best_fraction_index = np.argmin(mse)
    best_fraction = fractions[best_fraction_index]
    best_matches.append(best_fraction)

# Add the best match to the DataFrame
sample_data['Best_Match_%Meth'] = np.array(best_matches) 

# Create a new DataFrame for fractional contributions
fractional_contributions = pd.DataFrame({
    'Sample': sample_data['Name'],
    'Methane-cycling': sample_data['Best_Match_%Meth'],
    'Planktonic': 1- sample_data['Best_Match_%Meth']
})

# Display the results
print("Updated Sample Data:")
print(sample_data)

print("\nFractional Contributions:")
print(fractional_contributions)








Updated Sample Data:
  Name        G0        G1        G2        G3        G4      Cren        G5  \
0   S1  0.389506  0.097763  0.157369  0.042605  0.000274  0.306266  0.006217   
1   S4  0.481303  0.112187  0.158765  0.021725  0.003057  0.219767  0.003197   
2   S5  0.435868  0.157116  0.226899  0.031261  0.002502  0.144034  0.002321   
3   S7  0.416644  0.092921  0.127931  0.056481  0.008502  0.292885  0.004636   
4   S9  0.302591  0.107944  0.248903  0.116816  0.015371  0.204646  0.003729   
5  S10  0.272736  0.108790  0.270738  0.134849  0.019270  0.190468  0.003149   
6  S11  0.251405  0.116905  0.282700  0.168619  0.026520  0.151434  0.002418   

   Best_Match_%Meth  
0              0.33  
1              0.39  
2              0.59  
3              0.30  
4              0.60  
5              0.65  
6              0.73  

Fractional Contributions:
  Sample  Methane-cycling  Planktonic
0     S1             0.33        0.67
1     S4             0.39        0.61
2     S5             

In [5]:
#convert % contribution to fractions

fract_cont= fractional_contributions[['Methane-cycling', 'Planktonic']].to_numpy() 


In [8]:
# biphytane endmember stoichiometry using best fit stoichiometry and gdgt distributions from Keller et al., 2024
best_fit_stoichiometry =  np.array([
        [2, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [0.5, 1, 0.5, 0, 0, 0],
        [0, 1, 1, 0, 0, 0],
        [0.5, 0, 1, 0, 0.5, 0],
        [0, 0, 1, 0, 0, 1],
    ])

biphytane_stoich = 0.5*np.dot(gdgt_distribution,best_fit_stoichiometry)
biphytane_stoich

array([[0.5125, 0.32  , 0.16  , 0.    , 0.0125, 0.005 ],
       [0.5025, 0.05  , 0.2225, 0.    , 0.    , 0.215 ]])

In [9]:
#calculate relative abundances of biphytanes per samples
rel_abund = np.dot(fract_cont,biphytane_stoich)

In [10]:
#define the d2h optimized endmember values
d2h_plank = -308
d2h_meth = -222


In [11]:

# Calculate d2H Bulk Matrix
def calculate_d2h_bulk(fract_cont, biphytane_stoich, rel_abund, d2h_meth, d2h_plank):
    num_samples, num_biphytanes = rel_abund.shape
    d2h_bulk = np.zeros((num_samples, num_biphytanes))

    for i in range(num_samples):
        for j in range(num_biphytanes):
            numerator = (
                fract_cont[i, 0] * d2h_meth * biphytane_stoich[0, j] +
                fract_cont[i, 1] * d2h_plank * biphytane_stoich[1, j]
            )
            d2h_bulk[i, j] = numerator / (rel_abund[i, j] + 1e-10)  # Prevent division by zero
    
    return d2h_bulk

# Compute d2H Bulk Matrix
d2h_bulk_matrix = calculate_d2h_bulk(fract_cont, biphytane_stoich, rel_abund, d2h_meth, d2h_plank)

In [12]:
d2h_bulk_matrix

array([[-279.24406875, -242.71171801, -285.50687292,   -0.        ,
        -221.99999462, -307.02608078],
       [-274.05598336, -238.88989038, -280.91406926,   -0.        ,
        -221.99999545, -306.74004485],
       [-256.8508064 , -230.4233157 , -264.26451164,   -0.        ,
        -221.99999699, -305.21514785],
       [-281.84272991, -244.97709905, -287.73987716,   -0.        ,
        -221.99999408, -307.15131559],
       [-255.99410024, -230.11320744, -263.37297283,   -0.        ,
        -221.99999704, -305.10112325],
       [-251.71561881, -228.67405755, -258.82336756,   -0.        ,
        -221.99999727, -304.43949006],
       [-244.88750486, -226.69850254, -251.20961117,   -0.        ,
        -221.99999757, -302.91247925]])